In [2]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten

from tensorflow.keras import backend
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

from tensorflow.keras.initializers import glorot_normal
from tensorflow import random_normal_initializer
from tensorflow.keras.layers import Dropout

import numpy as np
import os
import random
import tensorflow as tf
from imageio import imsave
from skimage import color

from tensorflow.keras.initializers import he_normal
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import BatchNormalization,Concatenate
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array,array_to_img, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Get images
images = img_to_array(load_img('/content/drive/MyDrive/Coloring-greyscale-images-master/Alpha-version/0012304.png'))
images = np.array(images, dtype=float)

In [5]:
test_images = img_to_array(load_img('/content/drive/MyDrive/Coloring-greyscale-images-master/Alpha-version/0068528.png'))
test_images = np.array(images, dtype=float)

In [6]:
X_test_lable = rgb2lab(1.0/255*test_images)[:,:,0]
Y_test_lable = rgb2lab(1.0/255*test_images)[:,:,1:]
Y_test_lable /= 128

In [47]:
X_test_lable = X_test_lable.reshape(1, 256, 256, 1)
Y_test_lable = Y_test_lable.reshape(1, 256, 256, 2)

In [7]:
images.shape


(256, 256, 3)

In [8]:
X_lable = rgb2lab(1.0/255*images)[:,:,0]
Y_lable = rgb2lab(1.0/255*images)[:,:,1:]
Y_lable /= 128

In [9]:
X_lable = X_lable.reshape(1, 256, 256, 1)
Y_lable = Y_lable.reshape(1, 256, 256, 2)

In [10]:
# Building the neural network
model_cnn1 = Sequential()
model_cnn1.add(InputLayer(input_shape=(None, None, 1)))
model_cnn1.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model_cnn1.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model_cnn1.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model_cnn1.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model_cnn1.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model_cnn1.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model_cnn1.add(UpSampling2D((2, 2)))
model_cnn1.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model_cnn1.add(UpSampling2D((2, 2)))
model_cnn1.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model_cnn1.add(UpSampling2D((2, 2)))
model_cnn1.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [11]:
# Finish model
model_cnn1.compile(optimizer='rmsprop',loss='mse')

In [12]:
model_cnn1.fit(x=X_lable, y=Y_lable,batch_size=1,epochs=1000)

Epoch 1/1000
1/1 [==============================] - 9s 9s/step - loss: 0.5398
Epoch 2/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.5623
Epoch 3/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.4916
Epoch 4/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.0169
Epoch 5/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.0226
Epoch 6/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.0112
Epoch 7/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.0082
Epoch 8/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.0074
Epoch 9/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.0071
Epoch 10/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.0069
Epoch 11/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.0067
Epoch 12/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.0066
Epoch 13/1000
1/1 [=========================

In [14]:
print(model_cnn1.evaluate(X_lable, Y_lable, batch_size=1))
op = model_cnn1.predict(X_lable)
op *= 128
current = np.zeros((256, 256, 3))
current[:,:,0] = X_lable[0][:,:,0]
current[:,:,1:] = op[0]


1/1 [==============================] - 0s 34ms/step - loss: 0.0014
0.0013979070354253054
1/1 [==============================] - 0s 52ms/step


In [15]:
imsave("result_test.png", (lab2rgb(current)*255).astype(np.uint8))
imsave("grey_result_test.png", rgb2gray(lab2rgb(current)))

In [14]:
current.max()

94.15847285317675